In [1]:
import pandas as pd
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch
torch.cuda.get_device_name(0)

'NVIDIA GeForce MX250'

In [2]:
df = pd.read_csv('data\\traindata.csv',sep='\t',header=None)

In [3]:
df.columns = ['Polarity','Aspect_category','Aspect_term','Character_offset','Text']

In [4]:
def extract_thematique(text):
    return text.split('#')[0]

def extract_sujet(text):
    return text.split('#')[1]

In [5]:
df['thematique'] = df.Aspect_category.apply(extract_thematique)
df['sujet'] = df.Aspect_category.apply(extract_sujet)

In [6]:
thematique_names = list(df.thematique.unique())
thematique_names.sort()
thematiques_dico = {thematique_names[i]:i for i in range(len(thematique_names))}

In [7]:
sujet_names = list(df.sujet.unique())
sujet_names.sort()
sujet_dico = {sujet_names[i]:i for i in range(len(sujet_names))}

In [8]:
def create_vector_thematique(row):
    theme = row.thematique
    result = np.zeros(len(thematique_names),dtype=int)
    result[thematiques_dico[theme]] = 1
    return result

def create_vector_sujet(row):
    sujet = row.sujet
    result = np.zeros(len(sujet_names),dtype=int)
    result[sujet_dico[sujet]] = 1
    return result

In [9]:
df['thematique'] = df.Aspect_category.apply(extract_thematique)
df['sujet'] = df.Aspect_category.apply(extract_sujet)

In [10]:
thematique_names = list(df.thematique.unique())
thematique_names.sort()
thematiques_dico = {thematique_names[i]:i for i in range(len(thematique_names))}

In [11]:
sujet_names = list(df.sujet.unique())
sujet_names.sort()
sujet_dico = {sujet_names[i]:i for i in range(len(sujet_names))}

In [12]:
df['vector_thematique'] = df.apply(create_vector_thematique,axis=1)
df['vector_sujet'] = df.apply(create_vector_sujet,axis=1)

In [13]:
df.head()

,Polarity,Aspect_category,Aspect_term,Character_offset,Text,thematique,sujet,vector_thematique,vector_sujet
0,positive,AMBIENCE#GENERAL,seating,18:25,short and sweet – seating is great:it's romant...,AMBIENCE,GENERAL,"[1, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0]"
1,positive,AMBIENCE#GENERAL,trattoria,25:34,This quaint and romantic trattoria is at the t...,AMBIENCE,GENERAL,"[1, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0]"
2,positive,FOOD#QUALITY,food,98:102,The have over 100 different beers to offer thi...,FOOD,QUALITY,"[0, 0, 1, 0, 0, 0]","[0, 0, 0, 1, 0]"
3,negative,SERVICE#GENERAL,STAFF,5:10,THIS STAFF SHOULD BE FIRED.,SERVICE,GENERAL,"[0, 0, 0, 0, 0, 1]","[1, 0, 0, 0, 0]"
4,positive,FOOD#STYLE_OPTIONS,menu,4:8,"The menu looked great, and the waiter was very...",FOOD,STYLE_OPTIONS,"[0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 1]"


In [14]:
df.replace({'positive':1,'negative':-1,'neutral':0},inplace=True)

C:\Users\pauli\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\array_algos\replace.py:84: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)


In [15]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

def tokenize(row):
    return tokenizer(row.Text,padding='max_length',truncation=True)

In [16]:
df['tokenize'] = df.apply(tokenize,axis=1)
df['input_ids'] = df.tokenize.apply(lambda dico : dico['input_ids'])
df['token_type_ids'] = df.tokenize.apply(lambda dico : dico['token_type_ids'])
df['attention_mask'] = df.tokenize.apply(lambda dico : dico['attention_mask'])

In [17]:
class My_Dataset(Dataset):
    def __init__(self, df, transform = None):
    
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        vector_sujet = self.df['vector_sujet'].iloc[index]
        vector_thematique = self.df['vector_thematique'].iloc[index]
        input_ids = self.df['input_ids'].iloc[index]
        token_type_ids = self.df['token_type_ids'].iloc[index]
        attention_mask = self.df['attention_mask'].iloc[index]
        label = self.df['Polarity'].iloc[index]

        return {'sujet':torch.tensor(vector_sujet), 
                'thematique':torch.tensor(vector_thematique), 
                'input_ids': torch.tensor(input_ids), 
                'token_type_ids': torch.tensor(token_type_ids), 
                'attention_mask':torch.tensor(attention_mask),
                'label':torch.tensor(label)}

In [18]:
dataset = My_Dataset(df)
data_loader = DataLoader(dataset, batch_size=1)

In [19]:
for batch in data_loader:
    print(batch)
    break

{'sujet': tensor([[1, 0, 0, 0, 0]], dtype=torch.int32), 'thematique': tensor([[1, 0, 0, 0, 0, 0]], dtype=torch.int32), 'input_ids': tensor([[  101,  1603,  1105,  4105,   782, 11051,  1110,  1632,   131,  1122,
           112,   188,  6376,   117,  1884,  6482,  1105,  2029,   119,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,

In [20]:
from transformers import BertModel
import torch.nn as nn
from transformers import AutoModelForSequenceClassification


class CustomBERTModel(nn.Module):
    def __init__(self, bert_out_size = 24):
          super(CustomBERTModel, self).__init__()
          bert_model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased',num_labels=bert_out_size)
          self.bert = bert_model
          ### New layers:
          self.linear1 = nn.Linear(bert_out_size, 1)
          self.bert_out_size= bert_out_size
        

    def forward(self, input_ids, token_type_ids, attention_mask, sujet, thematique):
          bert_out = self.bert(
              input_ids=input_ids,
              token_type_ids=token_type_ids,
              attention_mask=attention_mask
            )
          bert_out = bert_out['logits']

          # sequence_output has the following shape: (batch_size, sequence_length, 768)
          linear1_output = self.linear1(bert_out.view(-1,self.bert_out_size)) ## extract the 1st token's embeddings
          
          return nn.Tanh(linear1_output)

In [21]:
model = CustomBERTModel()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [22]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [23]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(data_loader)
lr_scheduler = get_scheduler(name="linear", 
                             optimizer=optimizer, 
                             num_warmup_steps=0, 
                             num_training_steps=num_training_steps
)

In [24]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

CustomBERTModel(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=Tr

In [26]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(3))

for epoch in range(num_epochs):
    for batch in data_loader:
        labels = batch['label']
        batch = {k: v.to(device) for k, v in batch.items() if k != 'label'}
        
        outputs = model(**batch)

        print(outputs)
        print(outputs.logits.shape)

        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/3 [02:22<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 2.00 GiB total capacity; 1.47 GiB already allocated; 0 bytes free; 1.48 GiB reserved in total by PyTorch)